In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv("sample_data/qoute_dataset.csv")

In [ ]:
df.head()

,quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


In [ ]:
quotes = df['quote']
quotes =quotes.str.lower()
quotes.head()

,quote
0,“the world as we have created it is a process ...
1,"“it is our choices, harry, that show what we t..."
2,“there are only two ways to live your life. on...
3,"“the person, be it gentleman or lady, who has ..."
4,"“imperfection is beauty, madness is genius and..."


In [ ]:
import string
translator = str.maketrans('', '', string.punctuation)
quotes = quotes.apply(lambda x: x.translate(translator))
display(quotes.head())


,quote
0,“the world as we have created it is a process ...
1,“it is our choices harry that show what we tru...
2,“there are only two ways to live your life one...
3,“the person be it gentleman or lady who has no...
4,“imperfection is beauty madness is genius and ...


In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
vocab_size = 10000

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(quotes)

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))
list(word_index.items())[:10]

8978


[('the', 1),
 ('you', 2),
 ('to', 3),
 ('and', 4),
 ('a', 5),
 ('i', 6),
 ('is', 7),
 ('of', 8),
 ('that', 9),
 ('it', 10)]

In [ ]:
squence = tokenizer.texts_to_sequences(quotes)

In [ ]:
quotes[0]

'“the world as we have created it is a process of our thinking it cannot be changed without changing our thinking”'

In [ ]:
squence[0]

[713,
 62,
 29,
 19,
 16,
 946,
 10,
 7,
 5,
 1156,
 8,
 70,
 293,
 10,
 145,
 12,
 809,
 104,
 752,
 70,
 2461]

In [ ]:
from typing import Sequence
X = []
y =  []

for seq in squence:
  for i in range(1, len(seq)):
    input_seq = seq[:i]
    output_seq = seq[i]
    X.append(input_seq)
    y.append(output_seq)

above code doing this types of x and y

In [ ]:
print("Here are a few examples from X (input sequences) and y (next word targets):")
for i in range(5):
    print(f"X[{i}]: {X[i]}\t\ty[{i}]: {y[i]}")



Here are a few examples from X (input sequences) and y (next word targets):
X[0]: [713]		y[0]: 62
X[1]: [713, 62]		y[1]: 29
X[2]: [713, 62, 29]		y[2]: 19
X[3]: [713, 62, 29, 19]		y[3]: 16
X[4]: [713, 62, 29, 19, 16]		y[4]: 946


In [ ]:
max_len_X = max(len(x) for x in X)

print(max_len_X)


745


 Perform pad_sequences on X

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_padding = pad_sequences(X, maxlen=max_len_X, padding='pre')


In [ ]:
X_padding

array([[   0,    0,    0, ...,    0,    0,  713],
       [   0,    0,    0, ...,    0,  713,   62],
       [   0,    0,    0, ...,  713,   62,   29],
       ...,
       [   0,    0,    0, ...,    9,   19, 1125],
       [   0,    0,    0, ...,   19, 1125,    3],
       [   0,    0,    0, ..., 1125,    3,  169]], dtype=int32)

In [ ]:
print("\nAnd for the padded sequences:")
for i in range(5):
    print(f"X_padding[{i}][-5:]: {X_padding[i][-5:]}\t\ty[{i}]: {y[i]}")


And for the padded sequences:
X_padding[0][-5:]: [  0   0   0   0 713]		y[0]: 62
X_padding[1][-5:]: [  0   0   0 713  62]		y[1]: 29
X_padding[2][-5:]: [  0   0 713  62  29]		y[2]: 19
X_padding[3][-5:]: [  0 713  62  29  19]		y[3]: 16
X_padding[4][-5:]: [713  62  29  19  16]		y[4]: 946


In [ ]:
y = np.array(y)

 Perform one hot coding on y

In [ ]:
from IPython.utils.sysinfo import num_cpus
from tensorflow.keras.utils import to_categorical

y_one_hot =  to_categorical(y,num_classes=vocab_size)



In [ ]:
y_one_hot.shape

(85271, 10000)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN ,LSTM, Dense

 SimpleRNN model  Implement

In [ ]:
Embedding_dim = 50
rnn_units = 128

rnn_model = Sequential()

rnn_model.add(
         Embedding(input_dim=vocab_size, output_dim=Embedding_dim)
)
rnn_model.add( SimpleRNN(units=rnn_units))

rnn_model.add(Dense(units=vocab_size, activation='softmax'))

In [ ]:
rnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
rnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

 LSTM model  Implement


In [ ]:
lstm_model = Sequential()
lstm_model.add(
     Embedding(input_dim=vocab_size, output_dim=Embedding_dim, input_length=max_len_X
         )
)


lstm_model.add(LSTM(units=128))
lstm_model.add(Dense(units=vocab_size, activation='softmax'))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
lstm_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 10
batch_size = 128

In [ ]:
# history_rnn = rnn_model.fit(
#     X_padding, y_one_hot,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=0.1,
#     )


In [ ]:
epocs = 100
batch_size = 128

# from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(
#     monitor='val_loss',  # Monitor the validation loss
#     patience=5,          # Number of epochs with no improvement after which training will be stopped
#     restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
# )

history_lstm = lstm_model.fit(
    X_padding, y_one_hot,
    batch_size=batch_size,
    epochs=epocs,
    validation_split=0.1,
    # callbacks=[early_stopping] # Add the early stopping callback here
    )

Epoch 1/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 32s 52ms/step - accuracy: 0.1185 - loss: 5.4471 - val_accuracy: 0.1025 - val_loss: 6.6213
Epoch 2/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 30s 51ms/step - accuracy: 0.1237 - loss: 5.2905 - val_accuracy: 0.1078 - val_loss: 6.6898
Epoch 3/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 0.1323 - loss: 5.1211 - val_accuracy: 0.1066 - val_loss: 6.7770
Epoch 4/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1380 - loss: 4.9894 - val_accuracy: 0.1085 - val_loss: 6.8487
Epoch 5/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1433 - loss: 4.8545 - val_accuracy: 0.1112 - val_loss: 6.9058
Epoch 6/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 0.1512 - loss: 4.7362 - val_accuracy: 0.1119 - val_loss: 6.9684
Epoch 7/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1584 - loss: 4.6022 - val_accuracy: 0.1135 - val_loss: 7.0560
Epoch 8/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.1696 - loss: 4

In [ ]:
lstm_model.save('lstm_model.h5')

In [ ]:
import pickle as pickal

with open('tokenizer.pkl', 'wb') as f:
    pickal.dump(tokenizer,  f)

with open('max_len_X.pkl', 'wb') as f:
    pickal.dump(max_len_X,f)